## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import openpyxl
import math

## Access input and output directories 

In [2]:
#Access input folder
input_dir1 = Path ("Pressure_curves_excel_with_MinTime")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

1 Pressure_curves_excel_with_MinTime
2 [WindowsPath('Pressure_curves_excel_with_MinTime/L13_1'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_11'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_13'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_15'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_17'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_19'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_21'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_23'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_25'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_27'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_29'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_3'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_31'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_33'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_35'), WindowsPath('Pressure_curves_excel_with_MinTime/L13_37'), WindowsPath('Pressure_curves_excel

## Define find_end_time function

In [15]:
def find_end_time (Min_index, sh, stand_dis):
    if stand_dis == 1:
        for j in range(sh.max_row, Min_index, -1):
            cell_pressure = sh.cell (row = j, column = 2)
            if float(cell_pressure.value) < 101200:
                val_set=[sh.cell(row=k,column=2).value for k in range(j-1,j-21,-1)]
            
                if all(l < 101200 for l in val_set) == True:
                    print ("End_pressure1", cell_pressure.value)
                    end_time = sh.cell (row = j, column = 1)
                    print ("End_time1", end_time.value)
                    end_index = j
                    break
                
    else:
        for j in range(Min_index, sh.max_row+1):
            cell_pressure = sh.cell (row = j, column = 2)
            if float(cell_pressure.value) >= 101200:
                print ("standoff dis2", stand_dis)
                val_set=[sh.cell(row=k,column=2).value for k in range(j+1,j+149)]
            
                if all(l >= 101100 for l in val_set) == True:
                
                    print ("End_pressure_other", cell_pressure.value)
                    end_time = sh.cell (row = j, column = 1)
                    print ("End_time_other", end_time.value)
                    end_index = j

                    break
    
    return end_time, end_index

## Filtering end time

In [16]:
# Create empty dataframe
df = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'End_time', 'End_index'])

# Access folders
for folder in input_dir2:
    print ("folder", folder)
    
    # Set charge mass
    charge_mass = int(folder.stem[4:])/2
    
    # Make a list of data file names
    files = list(folder.rglob("*.xlsx"))
    
    # Access files inside folder
    for file in files:
        print (file.stem)
    
        wrkbk = openpyxl.load_workbook(file)
        sh = wrkbk.active
        
        # Set standoff distance
        stand_dis = math.ceil(int(file.stem[1:])/2)
        
        Min_time = sh.cell(row = 2, column = 3).value
        print ("Min_time_file_defined", Min_time, type(Min_time))
        
        if Min_time > 0.1:
            print ("Min_time into -Min_time > 0.1- loop", Min_time)
            pressure_values = [cell.value for cell in sh["B"]][1:]
            Min_pressure = min(pressure_values)
            Min_index = pressure_values.index(Min_pressure)
            
            print ("Min_index resulted from -Min_time > 0.1- loop", Min_index)
            Min_time = sh.cell(row = Min_index+2, column = 1).value
            print ("Min_time resulted from -Min_time > 0.104- loop", Min_time)
            
            end_time, end_index = find_end_time (Min_index, sh, stand_dis)
            
        else:
            Min_index = 2
            print ("Min_index = 2", Min_index)
            
            for i in range(2, sh.max_row+1):
                Time = sh.cell (row = i, column = 1)
                if Time.value > Min_time:
                    print ("Time > Min_time", Time.value)
                    Min_index = i
                    print ("Min_time_row after loop", i)
                    break
            
            end_time, end_index = find_end_time (Min_index, sh, stand_dis)
        
        # append rows to an empty DataFrame
        new_row = {'Charge_mass' : [charge_mass],
                   'Standoff_distance' : [stand_dis],
                   'End_time' : [end_time.value],
                   'End_index' : [end_index]}
        new_row_data = pd.DataFrame(new_row)
        df = pd.concat([df, new_row_data], ignore_index = True)

folder Pressure_curves_excel_with_MinTime\L13_1
P1
Min_time_file_defined 0.0023097852 <class 'float'>
Min_index = 2 2
Time > Min_time 0.0023097852245
Min_time_row after loop 233
End_pressure1 101194.36719
End_time1 0.011039755307
P13
Min_time_file_defined 0.0212596375 <class 'float'>
Min_index = 2 2
Time > Min_time 0.02125963755
Min_time_row after loop 2128
standoff dis2 7
End_pressure_other 101201.03906
End_time_other 0.028499947861
P17
Min_time_file_defined 0.0273598284 <class 'float'>
Min_index = 2 2
Time > Min_time 0.027369709685
Min_time_row after loop 2739
standoff dis2 9
End_pressure_other 101200.03906
End_time_other 0.034469928592
P21
Min_time_file_defined 0.0334397219 <class 'float'>
Min_index = 2 2
Time > Min_time 0.033439721912
Min_time_row after loop 3346
standoff dis2 11
End_pressure_other 101200.58594
End_time_other 0.040459807962
P25
Min_time_file_defined 0.0395098701 <class 'float'>
Min_index = 2 2
Time > Min_time 0.039509870112
Min_time_row after loop 3953
standoff dis

In [17]:
print (df.head(20))

    Charge_mass Standoff_distance  End_time End_index
0           0.5                 1   0.01104      1106
1           0.5                 7   0.02850      2852
2           0.5                 9   0.03447      3449
3           0.5                11   0.04046      4048
4           0.5                13   0.04644      4646
5           0.5                15   0.05243      5245
6           0.5                17   0.05842      5844
7           0.5                19   0.06440      6442
8           0.5                21   0.07039      7041
9           0.5                23   0.07638      7640
10          0.5                25   0.08236      8238
11          0.5                 3   0.01660      1662
12          0.5                27   0.08835      8837
13          0.5                29   0.09434      9436
14          0.5                30   0.09733      9735
15          0.5                 5   0.02253      2255
16          5.5                 1   0.02471      2473
17          5.5             

## Create End_time excel file

In [18]:
with pd.ExcelWriter('End_time.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)